In [3]:
import pandas as pd
import pickle as pk
import numpy as np
import dgl
import torch as th

In [98]:
NUM_EPIC_NOUN_CLASS = 352
NUM_EPIC_VERB_CLASS = 125
NUM_EPIC_USER = 32
KEY_EDGE_INTENSITY = 'intensity'


In [72]:

pkl = '/home/k1896871/Proj/gcn-master/gcn/EPIC_train_action_labels.pkl'
name_n_list = []
dic_participants_info = {}
with open(pkl, 'rb') as f:
    meta = pk.load(f)

    # 39596 for the whole dataset
    # there are 28472 action segments for training
    # 2513 unique actions

   
    li_activities = []
    li_actions = []
    tmp_participant = 'P01'
    tmp_activity = 'P01_01'
   
    for index, data in meta.iterrows():
        v = data['verb_class']
        n = data['noun_class']
        
        name_n = data['noun']
        name_n_list.append(name_n)
        vn = tuple((v,n))

        participant = data['participant_id']
        activity = data['video_id']

        if tmp_activity == activity:
            li_actions.append(vn)
            continue

        else:
            
            li_activities.append(li_actions)
            # dic_participant[activity[4:]] = li_actions
            tmp_activity = activity
            li_actions = []
            li_actions.append(vn)

        if tmp_participant != participant:

            dic_participants_info[tmp_participant] = li_activities
            li_activities = []
            tmp_participant = participant

In [74]:
print(name_n_list)

['door', 'light', 'door', 'fridge', 'celery', 'container', 'tofu', 'fridge', 'fridge', 'carrot', 'drawer', 'fridge', 'vegetable', 'cupboard', 'board:cutting', 'board:cutting', 'cupboard', 'drawer', 'knife', 'knife', 'knife', 'drawer', 'knife', 'tap', 'courgette', 'courgette', 'carrot', 'carrot', 'tap', 'vegetable', 'cupboard', 'grater', 'pan', 'pan', 'cupboard', 'cupboard', 'courgette', 'courgette', 'hob', 'courgette', 'courgette', 'courgette', 'courgette', 'courgette', 'courgette', 'courgette', 'courgette', 'drawer', 'drawer', 'spatula', 'courgette', 'salt', 'salt', 'salt', 'salt', 'courgette', 'spatula', 'celery', 'celery', 'tap', 'celery', 'tap', 'celery', 'celery', 'celery', 'celery', 'board:cutting', 'celery', 'celery', 'fridge', 'celery', 'fridge', 'spatula', 'spatula', 'spatula', 'container', 'onion', 'onion', 'onion', 'container', 'spatula', 'knife', 'onion', 'onion', 'onion', 'knife', 'kettle', 'tap', 'water', 'water', 'tap', 'kettle', 'spatula', 'vegetable', 'vegetable', 'gla

In [87]:
# initial Graph
G = dgl.DGLGraph()

G.add_nodes(NUM_EPIC_NOUN_CLASS,{'x':th.zeros((NUM_EPIC_NOUN_CLASS,5))})
print(G.edata)
print(G.has_edge_between(8,113))

p_keys = dic_participants_info.keys()
print(p_keys)
idx_key = 0
for p_id in p_keys:
    activity_list = dic_participants_info[p_id]
    idx_key += 1
    for actions in activity_list:
        
        if len(actions) != 0:
           
            ns = np.array(actions)[:,1]
        else:
            print(p_id)
#         print (ns)
        num_node = ns.size
       
        u = ns[0]
        tmp = th.zeros([1,NUM_EPIC_USER])
       
        for i in range(1,num_node):
            v = ns[i]
            print(u,v)
            print(G.has_edge_between(u,v))
            if not G.has_edge_between(u,v):

             
                G.add_edges(u,v,{KEY_EDGE_INTENSITY:th.zeros([1,NUM_EPIC_USER])})
                tmpM = G.edges[u,v].data[KEY_EDGE_INTENSITY]
                onehotValue = th.eye(NUM_EPIC_USER)[idx_key-1]
                G.edges[u,v].data[KEY_EDGE_INTENSITY] = tmpM + onehotValue
            
            else:

                tmpM = G.edges[u,v].data[KEY_EDGE_INTENSITY]
                onehotValue = th.eye(NUM_EPIC_USER)[idx_key-1]
                G.edges[u,v].data[KEY_EDGE_INTENSITY] = tmpM + onehotValue
                # print(G.edges[u,v].data[KEY_EDGE_INTENSITY])
               
                # print(G.edges[u,v].data[KEY_EDGE_INTENSITY])
            u = v



{}
False
dict_keys(['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P10', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30'])
8 113
False
113 8
False
8 10
False
10 185
False
185 29
False
29 159
False
159 10
False
10 10
False
10 40
False
40 9
False
9 10
False
10 90
False
90 8
False
8 19
False
19 19
False
19 8
False
8 9
False
9 5
False
5 5
False
5 5
True
5 9
False
9 5
True
5 3
False
3 69
False
69 69
False
69 40
False
40 40
False
40 3
False
3 90
False
90 8
True
8 131
False
131 1
False
1 1
False
1 8
False
8 8
False
8 69
False
69 69
True
69 47
False
47 69
False
69 69
True
69 69
True
69 69
True
69 69
True
69 69
True
69 69
True
69 69
True
69 9
False
9 9
False
9 22
False
22 69
False
69 31
False
31 31
False
31 31
True
31 31
True
31 69
False
69 22
False
22 185
False
185 185
False
185 3
False
3 185
False
185 3
True
3 185
True
185 185
True
185 185
True
185 185
True
185 19
False
19 185
False
185 185
True
185 10
Fals

In [71]:
print( G.edges[38,17].data[KEY_EDGE_INTENSITY])
print( G.edges[38,17].data.keys())
sum = G.edges[38,17].data[KEY_EDGE_INTENSITY].sum(1)
print(sum)
print(G.edata)

tensor([[5., 3., 1., 1., 7., 4., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         3., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0.]])
KeysView({'intensity': tensor([[5., 3., 1., 1., 7., 4., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         3., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0.]])})
tensor([29.])
{'intensity': tensor([[2., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}


In [94]:
TG = dgl.DGLGraph()

test_actions = dic_participants_info['P01'][0]
test_nouns = np.array(test_actions)[:,1]
print(test_nouns)

num_node = ns.size
TG.add_nodes(NUM_EPIC_NOUN_CLASS) # Must cord with the whole class number??
# As the idex itsef is the class id, but another problem. no it's more convinient
# as we don't need to add the noun label which is 
u = ns[0]
print(TG.edata)
print(TG.has_edge_between(8,113),'sdkjfklsjfklj')

for i in range(1,num_node):
    v = ns[i]
    # print(u,v)
    # print(TG.has_edge_between(u,v))
    if not TG.has_edge_between(u,v):

     
        TG.add_edges(u,v,{KEY_EDGE_INTENSITY:th.zeros([1,1])})
        tmpM = TG.edges[u,v].data[KEY_EDGE_INTENSITY]
        
        TG.edges[u,v].data[KEY_EDGE_INTENSITY] = tmpM + th.ones([1,1])
     
    else:

        tmpM = TG.edges[u,v].data[KEY_EDGE_INTENSITY]
        
        TG.edges[u,v].data[KEY_EDGE_INTENSITY] = tmpM + th.ones([1,1])
        # print(G.edges[u,v].data[KEY_EDGE_INTENSITY])
       
        # print(G.edges[u,v].data[KEY_EDGE_INTENSITY])
    u = v


[  8 113   8  10 185  29 159  10  10  40   9  10  90   8  19  19   8   9
   5   5   5   9   5   3  69  69  40  40   3  90   8 131   1   1   8   8
  69  69  47  69  69  69  69  69  69  69  69   9   9  22  69  31  31  31
  31  69  22 185 185   3 185   3 185 185 185 185  19 185 185  10 185  10
  22  22  22  29  13  13  13  29  22   5  13  13  13   5  38   3  17  17
   3  38  22  90  90  16  16   8  16   8  23   8  23  29  29  29  29  11
  29   4   4   9   9   9   9  18  18  22  90  29  10  29  10  22  90 131
 131  40  40 131  40  40  40  40 131  40 131 131  40  22  90   8   1   1
  47  38  17  17  38  24  24  24  24  22  19  13  19  90  90  31  31  31
   9   7  31  31  17  31   7  31  22  90  47  90  90  90  22   8   8   8
  44  44  44  22  90  90  22 159   5 159  29 159  29  90  22  29  29  29
  29  22  90  44  44  44  20  20   8  44  90  22  28  78  17  78  28  75
   8  75   9  22  90  90  90  44  22   7  16  16  16   9  22  90  22  29
  29  29  29  29  22  44  90  90  22  90  44  22  2

In [111]:
intensity = TG.edata[KEY_EDGE_INTENSITY]

index = intensity.max(0)
print(index)
# print(intensity[index])

(tensor([24.]), tensor([15]))
